# **Project Scope**  
The primary objective is to **optimize inventory and purchasing management to reduce overstocking by 20%** within six months. The two key target variables for this project are Stock_Quantity for inventory optimization and Sales_Volume for demand forecasting.


## Project Scope Breakdown
**Inventory Optimization** (Target: `Stock_Quantity`)  

This phase focuses on the `Stock_Quantity` variable, which represents the total inventory units held daily. The goal is to analyze its patterns, such as replenishment and consumption cycles, to identify opportunities to reduce excess stock. The ultimate objective is a **20% reduction in overstocking**. The insights gained here will be critical for developing effective inventory control strategies and supply chain planning. The provided dataframe columns like `Reorder_Level`, `Reorder_Quantity`, `Inventory_Turnover_Rate`, and `DOI_Inventory_Turnover` will be crucial for this analysis.
<br><br>

**Demand Forecasting** (Target: `Sales_Volume`)  

The second part of the project involves forecasting future demand using the `Sales_Volume` variable, which tracks total daily units sold. Analyzing this variable will help predict future sales and understand underlying patterns like weekly seasonality or spikes from promotions. Accurate demand forecasts are essential for making informed decisions about purchasing and inventory levels. This will directly support the main objective of reducing overstocking by ensuring that inventory levels are aligned with expected customer demand.

Import Libraries

In [22]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests, acf, pacf, adfuller, kpss

# Configure display and graphs
pd.set_option('display.max_columns', None)
pio.templates['custom'] = pio.templates["plotly_white"]
pio.templates['custom'].layout.update(width=600, height=400)
pio.templates.default = "custom"

import warnings
warnings.filterwarnings('ignore')

Load Data

In [2]:
# Define data paths
data_path = os.path.join('../data', 'processed')

# Load Pickle file
read_data = pd.read_pickle(data_path + '/grocery.pkl')

In [3]:
# Sort DataFrame by Date_Received in ascending order
read_data = read_data.sort_values(by='Date_Received').reset_index(drop=True)


# **Target Variables Analysis**

In [4]:
# These are the key metrics the model aims to predict.
target_columns = ['Stock_Quantity', 'Sales_Volume']

# These are predictors that are fixed or change very infrequently for each product.
stat_predict = ['Product_Name','Category', 'Supplier_Name', 'Reorder_Level', 'Reorder_Quantity']

# These are predictors that can change over time for each product.
dyn_predict = ['Unit_Price', 'Date_Received', 'Inventory_Turnover_Rate', 'Status', 'Days_For_Expiration', 'Expiration_Status', 'DOI_Inventory_Turnover','Delivery_Lag']

# The complete set of all columns used in the analysis.
all_columns = stat_predict + dyn_predict + target_columns

In [5]:
# Create a copy of the selected columns from the original DataFrame for time series analysis
df_ts = read_data[all_columns].copy()

In [6]:
# data Information
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Product_Name             990 non-null    object        
 1   Category                 990 non-null    category      
 2   Supplier_Name            990 non-null    object        
 3   Reorder_Level            990 non-null    int64         
 4   Reorder_Quantity         990 non-null    int64         
 5   Unit_Price               990 non-null    float64       
 6   Date_Received            990 non-null    datetime64[ns]
 7   Inventory_Turnover_Rate  990 non-null    int64         
 8   Status                   990 non-null    category      
 9   Days_For_Expiration      990 non-null    Int64         
 10  Expiration_Status        990 non-null    category      
 11  DOI_Inventory_Turnover   990 non-null    int64         
 12  Delivery_Lag             990 non-nul

In [7]:
# Aggregate target columns by date, summing values for each day
data = read_data.groupby('Date_Received')[target_columns].sum().reset_index()

# Set the date column as the DataFrame index for time series operations
df_target = data.set_index('Date_Received')

# Resample to daily frequency, ensuring all dates are represented and filling missing days with 0
df_target = df_target.resample('D').sum().fillna(0)

# **Data Structure Prepared**
- **Frequency:** Daily time series (resampled to 'D')
- **Index:** DateTime index (`Date_Received`)
- **Missing Values:** Zero-filled for days without recorded activity
- **Decomposition:** Additive model with 7-day period for weekly seasonality analysis

### Series Visualization

In [8]:
# Create a blank subplot figure with 2 vertical panels and titles
fig = make_subplots(rows=2, cols=1, subplot_titles=['Sales Volume', 'Stock Quantity'])

# Create individual line plots for each target variable
fig1 = px.line(df_target, y=['Sales_Volume'], color_discrete_sequence=['blue'])
fig2 = px.line(df_target, y=['Stock_Quantity'], color_discrete_sequence=['green'])

# Add the first plot (Sales Volume) to the top panel
fig.add_trace(fig1.data[0], row=1, col=1)
# Add the second plot (Stock Quantity) to the bottom panel
fig.add_trace(fig2.data[0], row=2, col=1)

# Adjust the overall figure layout (hide legend, set height)
fig.update_layout(showlegend=False)
# Display the interactive plot
fig.show()

#### **Analysis of Time Series**

##### **1. Sales Volume**
The plot for **`Sales Volume`** shows a consistent, cyclical pattern over the observed period (March 2024 to February 2025). The data fluctuates daily, indicating a dynamic sales environment. While there is no clear long-term upward or downward trend, the plot exhibits significant **volatility** with frequent peaks and troughs. These short-term fluctuations are typical for daily sales data and may be influenced by factors like **weekly seasonality** (e.g., higher sales on weekends) or specific promotional events. This is a critical insight for your **demand forecasting** model, which will need to be robust enough to capture these short-term variations accurately.

---

##### **2. Stock Quantity**
The time series for **`Stock Quantity`** displays a very similar pattern to the `Sales Volume` but with a slightly different behavior. The stock quantity fluctuates in response to both sales and replenishment activities. The consistent, albeit volatile, nature of this series suggests a well-managed but potentially reactive inventory system. It appears that stock levels are being adjusted frequently, likely based on short-term sales demand. The close resemblance between the two series implies a strong relationship: as sales increase, stock decreases, and then replenishment cycles cause the stock to rise again. To achieve your goal of reducing overstocking, a key area of focus should be on optimizing the **timing and size of these replenishment orders**, ensuring they are based on a more accurate forecast rather than just a reaction to current sales.

---

#### **Key Insights for Your Project**

* **Relationship between Variables:** The two time series are highly interdependent. `Sales Volume` is the primary driver of demand, while `Stock Quantity` is the result of managing that demand. * **Forecasting Strategy:** Your demand forecasting model should not only predict the overall sales trend but also capture the **daily and weekly fluctuations** seen in the `Sales Volume` data. This is crucial for precise inventory management.
* **Inventory Optimization:** The volatility in `Stock Quantity` suggests opportunities for improvement. By using a better demand forecast, you can proactively adjust inventory levels to avoid holding excess stock (overstocking) and reduce the risk of stockouts. The goal is to smooth out the `Stock Quantity` curve, making it more predictable and efficient. This will directly contribute to your target of a **20% reduction in overstocking**.

### Decomposition for Sales Volume and Stock Quantity

In [9]:
# Use an appropriate period, such as 7 for daily data with weekly seasonality
result_sales = seasonal_decompose(df_target['Sales_Volume'], model='additive', period=7)
result_stock = seasonal_decompose(df_target['Stock_Quantity'], model='additive', period=7)

# Create a new DataFrame with the decomposed components
decomposed_df = pd.DataFrame({
    'Trend_Sales': result_sales.trend,
    'Seasonality_Sales': result_sales.seasonal,
    'Residuals_Sales': result_sales.resid,
    'Trend_Stock': result_stock.trend,
    'Seasonality_Stock': result_stock.seasonal,
    'Residuals_Stock': result_stock.resid,
}, index=df_target.index)

# Plot each component individually with Plotly Express
fig_decomposed = make_subplots(rows=2, cols=1, 
                              subplot_titles=['Seasonal Decompose Sales Volume', 
                                            'Seasonal Decompose Stock Quantity'])

# Create individual plots for sales components
fig_decomposed1 = px.line(decomposed_df[['Trend_Sales', 'Seasonality_Sales', 'Residuals_Sales']])
# Create individual plots for stock components with custom colors
fig_decomposed2 = px.line(decomposed_df[['Trend_Stock', 'Seasonality_Stock', 'Residuals_Stock']], 
                         color_discrete_sequence=["#ffaf3c", "#3c5587", "#adb7c4"])

# Add sales components to top subplot
for trace in fig_decomposed1.data[:3]:
    fig_decomposed.add_trace(trace, row=1, col=1)
    
# Add stock components to bottom subplot    
for trace2 in fig_decomposed2.data[:3]:
    fig_decomposed.add_trace(trace2, row=2, col=1)

# Update layout settings
fig_decomposed.update_layout(showlegend=True, height=800)

# Display the interactive decomposition plot
fig_decomposed.show()

#### **Analysis of Seasonal Decomposition**

##### **1. Sales Volume Decomposition**

* **Trend Component (Trend_Sales):** The trend line (blue) for `Sales Volume` is relatively flat and stable throughout the year. This suggests there is **no significant long-term growth or decline** in sales. The consistent demand over time is a positive sign for business stability, but it also means that sales are not organically increasing, so growth must come from other strategies.

* **Seasonality Component (Seasonality_Sales):** The seasonality plot (red) shows a clear and consistent **weekly or sub-weekly pattern**. This is a crucial finding for your demand forecasting model. The regular peaks and valleys indicate that sales are predictable on a cyclical basis, likely corresponding to specific days of the week (e.g., higher sales on weekends). Your model must capture this pattern to provide accurate short-term forecasts.

* **Residuals Component (Residuals_Sales):** The residuals (green) represent the random noise or irregular fluctuations that are not explained by the trend or seasonality. The residuals plot is volatile, with spikes that could be caused by unpredictable events like promotions, holidays, or one-off sales. These are hard to forecast and represent the **inherent unpredictability** of the sales process.

---

##### **2. Stock Quantity Decomposition**

* **Trend Component (Trend_Stock):** The trend line (orange) for `Stock Quantity` mirrors the `Sales Volume` trend—it is relatively flat. This shows that your overall inventory levels are being maintained at a consistent average over time. This is a good baseline, but it also indicates there's room for optimization to reduce average stock levels.

* **Seasonality Component (Seasonality_Stock):** The seasonality plot (dark blue) shows a pattern that closely follows the sales seasonality. This is a significant finding. It suggests that your current inventory management system is **reacting to the seasonal demand**, with stock levels being adjusted cyclically in sync with sales.

* **Residuals Component (Residuals_Stock):** The residuals (grey) for `Stock Quantity` are highly volatile, with many large spikes. This volatility is a key indicator of **overstocking and understocking events**. These spikes represent times when inventory levels are either excessively high or critically low, often in a reactive manner. This is the primary area to address to achieve your **20% overstocking reduction goal**. By improving the demand forecast and using a more proactive inventory strategy, you can minimize these residual spikes.

#### **Conclusion**

The seasonal decomposition confirms that your data has clear, predictable **seasonal patterns**, which your forecasting model can leverage. However, the high volatility in the **residuals** for `Stock Quantity` indicates that the current inventory management system is reactive and inefficient. The project's success hinges on building a model that can accurately predict the seasonal sales patterns and then using that prediction to create a more stable and efficient **proactive inventory strategy**, thereby reducing the large spikes in the stock residuals.

### Stationarity Tests Analysis

In [23]:
from statsmodels.tsa.stattools import adfuller, kpss

def stationarity_tests(series, signif=0.05, name='', verbose=True):
    '''
        Combined Interpretation:

        Both tests reject H₀ → inconclusive result; there may be a trend or structural break.

        ADF rejects H₀ and KPSS does not reject H₀ → strong evidence of stationarity.

        ADF does not reject H₀ and KPSS rejects H₀ → strong evidence of non-stationarity.
    '''
    
    if series is None or len(series) == 0:
        raise ValueError("Input series is empty or None.")

    # ADF Test
    adf_result = adfuller(series, autolag='AIC')
    adf_stat, adf_p, adf_lags, adf_nobs, adf_crit, _ = adf_result

    # KPSS Test
    kpss_result = kpss(series, regression='c', nlags='auto')
    kpss_stat, kpss_p, kpss_lags, kpss_crit = kpss_result

    if verbose:
        print(f'Stationarity Tests on "{name}"\n')

        print('ADF Test (H₀: Non-stationary)')
        print(f'  Test Statistic      : {adf_stat:.4f}')
        print(f'  p-value             : {adf_p:.4f}')
        print(f'  # Lags Used         : {adf_lags}')
        print(f'  # Observations Used : {adf_nobs}')
        print('  Critical Values:')
        for key, val in adf_crit.items():
            print(f'    {key} : {val:.4f}')
        print(f'  => {"Reject" if adf_p <= signif else "Fail to reject"} H₀: Series is {"stationary" if adf_p <= signif else "not stationary"}\n')

        print('KPSS Test (H₀: Stationary)')
        print(f'  Test Statistic      : {kpss_stat:.4f}')
        print(f'  p-value             : {kpss_p:.4f}')
        print(f'  # Lags Used         : {kpss_lags}')
        print('  Critical Values:')
        for key, val in kpss_crit.items():
            print(f'    {key} : {val:.4f}')
        print(f'  => {"Reject" if kpss_p <= signif else "Fail to reject"} H₀: Series is {"not stationary" if kpss_p <= signif else "stationary"}\n')

    return {
        'ADF': {
            'statistic': adf_stat,
            'p_value': adf_p,
            'lags': adf_lags,
            'n_obs': adf_nobs,
            'critical_values': adf_crit,
            'stationary': adf_p <= signif
        },
        'KPSS': {
            'statistic': kpss_stat,
            'p_value': kpss_p,
            'lags': kpss_lags,
            'critical_values': kpss_crit,
            'stationary': kpss_p > signif
        }
    }


In [40]:
# Run stationarity tests (ADF and KPSS) on the 'Sales_Volume' time series
# The function returns a dictionary with detailed results from both tests
stat_tests_sales_volume = stationarity_tests(series=df_target['Sales_Volume'], name='Sales_Volume', verbose=False)

# Print a summary of the stationarity status based on both methods
# ADF: Null hypothesis is non-stationarity → if rejected, series is stationary
# KPSS: Null hypothesis is stationarity → if rejected, series is non-stationary
print(f'The time series of Sales Volume exhibits stationarity.\n'
      f'Method ADF: {stat_tests_sales_volume["ADF"]["stationary"]}\n'
      f'Method KPSS: {stat_tests_sales_volume["KPSS"]["stationary"]}')


The time series of Sales Volume exhibits stationarity.
Method ADF: True
Method KPSS: True


In [42]:
# Run stationarity tests (ADF and KPSS) on the 'Stock_Quantity' time series
# The function returns a dictionary with detailed results from both tests
stat_tests_Stock_Quantity = stationarity_tests(series=df_target['Stock_Quantity'], name='Stock_Quantity', verbose=False)

# Print a summary of the stationarity status based on both methods
# ADF: Null hypothesis is non-stationarity → if rejected, series is stationary
# KPSS: Null hypothesis is stationarity → if rejected, series is non-stationary
print(f'The time series of Stock Quantity exhibits stationarity.\n'
      f'Method ADF: {stat_tests_Stock_Quantity["ADF"]["stationary"]}\n'
      f'Method KPSS: {stat_tests_Stock_Quantity["KPSS"]["stationary"]}')


The time series of Stock Quantity exhibits stationarity.
Method ADF: True
Method KPSS: True


In [35]:
stat_tests_sales_volume

{'ADF': {'statistic': -4.383286760427548,
  'p_value': 0.000317714608687523,
  'lags': 14,
  'n_obs': 349,
  'critical_values': {'1%': -3.449226932880019,
   '5%': -2.869857365438656,
   '10%': -2.571201085130664},
  'stationary': True},
 'KPSS': {'statistic': 0.2036412009429863,
  'p_value': 0.1,
  'lags': 13,
  'critical_values': {'10%': 0.347, '5%': 0.463, '2.5%': 0.574, '1%': 0.739},
  'stationary': True}}

### Autocorrelation Analysis

In [10]:
# Function for interactive ACF/PACF charts
def plot_acf_pacf(series, title, max_lag=40):
    acf_values = acf(series.dropna(), nlags=max_lag)
    pacf_values = pacf(series.dropna(), nlags=max_lag)
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=[f'ACF - {title}', f'PACF - {title}'])
    
    # ACF
    fig.add_trace(go.Bar(x=list(range(max_lag+1)), y=acf_values, name='ACF'), row=1, col=1)
    fig.add_hline(y=1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=1)
    fig.add_hline(y=-1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=1)
    
    # PACF
    fig.add_trace(go.Bar(x=list(range(max_lag+1)), y=pacf_values, name='PACF'), row=1, col=2)
    fig.add_hline(y=1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=2)
    fig.add_hline(y=-1.96/np.sqrt(len(series)), line_dash="dash", row=1, col=2)
    
    fig.update_layout(height=400, showlegend=False, title_text=f"Autocorrelation Analysis - {title}")
    fig.show()

# Plot for Sales Volume
plot_acf_pacf(df_target['Sales_Volume'], 'Sales Volume')

# Plot for Stock Quantity
plot_acf_pacf(df_target['Stock_Quantity'], 'Stock Quantity')

#### **Autocorrelation Analysis of Time Series**

This analysis uses the **Autocorrelation Function (ACF)** and **Partial Autocorrelation Function (PACF)** to identify underlying patterns in your data, which is crucial for building accurate forecasting models.

##### **1. Sales Volume**

The **ACF plot** for `Sales Volume` shows a series of significant spikes at lags 7, 14, 21, and so on. This pattern is a clear indicator of **weekly seasonality**, meaning sales are strongly correlated with sales from the same day of the week in prior weeks. The **PACF plot** confirms this, showing significant spikes at these same lags after accounting for the influence of intervening days. This is great news, as it means the demand is predictable and well-suited for a seasonal time-series model.

---

##### **2. Stock Quantity**

The autocorrelation plots for **`Stock Quantity`** reveal a pattern that is nearly identical to that of `Sales Volume`.

* **ACF Plot:** Like the sales data, the ACF for `Stock Quantity` shows significant, decaying spikes at lags 7, 14, 21, and subsequent multiples of 7. This strong autocorrelation confirms a **weekly seasonal pattern** in your inventory levels.
* **PACF Plot:** The PACF plot also shows significant spikes at the weekly lags. This indicates that your inventory levels are not only affected by recent stock changes but are also directly influenced by the stock levels from the same day in previous weeks.

#### **Combined Insights for Your Project**

The identical patterns in the autocorrelation plots for both sales and stock quantity confirm a crucial point: **your current inventory management is highly reactive to weekly sales seasonality.**

Instead of being based on a long-term forecast, your system appears to be directly reacting to the predictable weekly fluctuations in demand. To achieve your goal of a **20% overstocking reduction**, you must shift from this reactive approach to a **proactive one**.

By using a seasonal forecasting model (like SARIMA) that leverages the weekly patterns you've identified, you can predict future sales with greater accuracy. This will allow you to optimize your reorder quantities and timing, reducing the need for excess stock and minimizing the costs associated with overstocking.

In [11]:
# Create new columns with previous day's values (lag of 1)
df_target['Sales_Volume_Lag1'] = df_target['Sales_Volume'].shift(1)
df_target['Stock_Quantity_Lag1'] = df_target['Stock_Quantity'].shift(1)

# Create subplot figure with 2 horizontal panels
fig_lag = make_subplots(rows=1, cols=2, 
                       subplot_titles=['Autocorrelation Sales Volume', 
                                     'Autocorrelation Stock Quantity'])

# Create scatter plots to visualize autocorrelation
fig_lag1 = px.scatter(df_target, x='Sales_Volume', y='Sales_Volume_Lag1',
                     title='Autocorrelation (Sales_Volume vs. Previous Day Sales)')
fig_lag2 = px.scatter(df_target, x='Stock_Quantity', y='Stock_Quantity_Lag1',
                     title='Autocorrelation (Stock_Quantity vs. Previous Day Stock)', 
                     color_discrete_sequence=['green'])

# Add plots to subfigure
fig_lag.add_trace(fig_lag1.data[0], row=1, col=1)
fig_lag.add_trace(fig_lag2.data[0], row=1, col=2)

# Adjust layout and display
fig_lag.update_layout(showlegend=False, height=600)
fig_lag.show()

Here is an analysis of the relationship between consecutive values for both **`Sales Volume`** and **`Stock Quantity`**. This type of plot, where each point represents (`Value_at_Time_t`, `Value_at_Time_t+1`), is another way to visualize autocorrelation.

#### **Analysis of Scatter Plots**

##### **1. Autocorrelation of Sales Volume**
The scatter plot for **`Sales Volume`** shows a distinct, albeit weak, positive linear relationship. The points are not tightly clustered, but there's a general trend: high sales one day tend to be followed by high sales the next day, and low sales by low sales. This confirms the **positive short-term autocorrelation** seen in the ACF/PACF plots. However, the wide scatter of the points also indicates a high degree of daily randomness or noise, which is expected in sales data.

##### **2. Autocorrelation of Stock Quantity**
The scatter plot for **`Stock Quantity`** displays a much **stronger and more positive linear relationship** than the sales data. The points are more clustered along a diagonal line, indicating that today's stock quantity is a very strong predictor of tomorrow's stock quantity. This is a crucial finding. It suggests that inventory levels are not fluctuating wildly from one day to the next. Instead, they are being maintained in a stable, predictable manner, likely due to a consistent replenishment and consumption cycle.

---

#### **Key Insights for Your Project**

The analysis of these scatter plots reinforces the findings from the previous plots.

* **Sales vs. Stock:** While sales volume is somewhat predictable, it also has a significant amount of daily unpredictability (the wide scatter). Your current inventory management system is not simply reacting to daily sales; rather, it's maintaining a more stable and less volatile stock level. * **Inventory Stability:** The strong autocorrelation in `Stock Quantity` is a double-edged sword. It confirms that your inventory levels are stable, which is a good sign of a well-run system. However, this stability might also be a symptom of **overstocking**. A high correlation means that your inventory levels aren't being adjusted dynamically enough to respond to real-time demand, which could lead to holding more stock than necessary.<br><br>
* **Optimization Opportunity:** The goal of reducing overstocking by 20% means you need to break this strong positive autocorrelation in `Stock Quantity` in a controlled way. Instead of just maintaining stable inventory, you need to use a more accurate demand forecast to make more precise, proactive adjustments. This will allow you to reduce your overall average stock level while still maintaining an adequate buffer against stockouts.

### Outliers

In [12]:
# Create a box plot for Sales_Volume and Stock_Quantity
fig_box = px.box(df_target, 
                 y=['Sales_Volume', 'Stock_Quantity'], 
                 title='Distribution of Sales Volume and Stock Quantity with Outlier Detection')
fig_box.update_layout(height=600)
fig_box.show()

#### Box Plot Analysis Summary

The box plot analysis confirms that the **`Sales Volume`** and **`Stock Quantity`** distributions are very similar in terms of their central tendencies. Both have a median value around **140-145 units**, indicating that, on average, your inventory levels are closely aligned with your daily sales. 

| Variable | Median (Approx.) | IQR (Approx.) | Key Finding |
| :--- | :--- | :--- | :--- |
| **Sales Volume** | 140 | 75 to 225 | High sales outliers exist, likely due to promotions. |
| **Stock Quantity** | 145 | 70 to 215 | **Overstocking outliers** are present, which is a direct target for your project. |

The key insight for your project is the presence of **outliers** in the `Stock Quantity` distribution. These points, representing days with unusually high inventory, are a visual manifestation of **overstocking**. The project's success in achieving a **20% reduction** will depend on a system that can effectively minimize these outliers by making more precise, data-driven inventory decisions.

### Casuality Test

In [13]:
# If the p-value is less than 0.05 (or your chosen significance level), you reject H_0.
# This means that series X causes series Y, i.e., past values of X help predict future values of Y.
# If the p-value is greater than 0.05, you fail to reject H_0.
# This means that series X is not useful for predicting Y.

# Granger Test: Does stock cause sales?
# H0: Stock does NOT cause sales.
gc_test_stock_to_sales = grangercausalitytests(df_target[['Sales_Volume', 'Stock_Quantity']], maxlag=7)
# The maxlag=7 argument tests causality for up to 7 days of lag,
# which is useful for identifying potential weekly relationships.

# Granger Test: Do sales cause stock?
# H0: Sales do NOT cause stock.
gc_test_sales_to_stock = grangercausalitytests(df_target[['Stock_Quantity', 'Sales_Volume']], maxlag=7)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.9721  , p=0.0470  , df_denom=360, df_num=1
ssr based chi2 test:   chi2=4.0052  , p=0.0454  , df=1
likelihood ratio test: chi2=3.9832  , p=0.0460  , df=1
parameter F test:         F=3.9721  , p=0.0470  , df_denom=360, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.6039  , p=0.0754  , df_denom=357, df_num=2
ssr based chi2 test:   chi2=5.2808  , p=0.0713  , df=2
likelihood ratio test: chi2=5.2426  , p=0.0727  , df=2
parameter F test:         F=2.6039  , p=0.0754  , df_denom=357, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.0502  , p=0.0287  , df_denom=354, df_num=3
ssr based chi2 test:   chi2=9.3317  , p=0.0252  , df=3
likelihood ratio test: chi2=9.2131  , p=0.0266  , df=3
parameter F test:         F=3.0502  , p=0.0287  , df_denom=354, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.1668  , p=0.0723  

In [14]:
# Analyze the results of the "stock to sales" test:
print("Granger Test: Does stock cause sales?")
for lag in range(1, 8):
    p_value = gc_test_stock_to_sales[lag][0]['ssr_ftest'][1]
    print(f"Lag {lag}: p-value = {p_value:.4f}")

# Analyze the results of the "sales to stock" test:
print("\nGranger Test: Do sales cause stock?")
for lag in range(1, 8):
    p_value = gc_test_sales_to_stock[lag][0]['ssr_ftest'][1]
    print(f"Lag {lag}: p-value = {p_value:.4f}")

Granger Test: Does stock cause sales?
Lag 1: p-value = 0.0470
Lag 2: p-value = 0.0754
Lag 3: p-value = 0.0287
Lag 4: p-value = 0.0723
Lag 5: p-value = 0.1085
Lag 6: p-value = 0.1944
Lag 7: p-value = 0.3293

Granger Test: Do sales cause stock?
Lag 1: p-value = 0.7229
Lag 2: p-value = 0.7164
Lag 3: p-value = 0.4047
Lag 4: p-value = 0.5852
Lag 5: p-value = 0.4488
Lag 6: p-value = 0.4622
Lag 7: p-value = 0.2250


#### Granger Causality Test Analysis

The Granger causality test is a statistical method to determine if one time series is useful in forecasting another. It doesn't imply true cause and effect but rather whether one variable's past values can help predict another's future values.

---

#### Test 1: "Does stock cause sales?"

The results show that at **lags 1 and 3**, the p-values are below the common significance threshold of 0.05. This means there is **significant evidence** to suggest that past `Stock_Quantity` values have a predictive influence on future `Sales_Volume`. This is an unusual finding. A possible explanation is that inventory levels are being managed in a way that directly impacts sales, such as by running a promotion when stock levels are high, which then increases sales. This indicates a reactive and potentially inefficient feedback loop in your supply chain.

---

#### Test 2: "Do sales cause stock?"

In this test, all p-values are well **above the 0.05 threshold**. This is the **most critical finding** for your project. It indicates there is **no significant evidence** that past `Sales_Volume` values are useful for predicting future `Stock_Quantity`. In a well-optimized supply chain, you would expect to see a strong causal link from sales to stock, as demand forecasts (based on sales history) should drive inventory decisions.

---

#### Conclusion

The Granger test results reveal a problematic one-way relationship: **stock quantity has a slight predictive influence on sales, but sales do not have a significant predictive influence on stock quantity.**

To achieve your goal of reducing overstocking by 20%, you must establish this missing link. The current system appears to be making inventory decisions reactively, leading to overstocking, as it isn't effectively using historical sales data to proactively manage inventory levels.

Proposal next steps should be to:
1.  **Build a robust demand forecasting model** using the `Sales_Volume` data, which you've already identified as having strong seasonality.
2.  **Integrate this forecast directly into your inventory management system** to drive purchasing and stocking decisions.

By doing so, you will create the predictive link from **sales to stock** that the Granger test shows is currently absent, allowing you to optimize inventory and reduce overstocking.

# **Analytical Value**
This prepared dataset enables:
- Trend analysis and seasonal pattern identification
- Correlation analysis between inventory levels and sales performance
- Forecasting model development for both inventory and demand planning
- Anomaly detection in stock movements and sales patterns

In [15]:
# Select numeric columns (int, float)
numeric_cols = df_ts.select_dtypes(include=[np.number]).columns.tolist()

# Select non-numeric columns (exclude numbers and datetime objects)
non_numeric_cols = df_ts.select_dtypes(exclude=[np.number, 'datetime']).columns.tolist()

In [16]:
# Calculate the correlation matrix for all numeric columns
correlation_matrix = df_ts[numeric_cols].corr()

In [17]:
# Create a heatmap visualization of the correlation matrix
fig_corr = px.imshow(correlation_matrix,
                    title='Correlation Matrix - Numeric Variables',
                    color_continuous_scale='RdBu_r',  # Red-Blue reversed color scale
                    aspect="auto",                   # Automatic aspect ratio
                    text_auto=True,                  # Display correlation values on cells
                    zmin=-1, zmax=1)                 # Fix color scale from -1 to +1

# Adjust the figure dimensions
fig_corr.update_layout(width=1000, height=1000)

# Display the interactive heatmap
fig_corr.show()

In [18]:
# Extract specific correlations with the target variables
correlation_with_targets = correlation_matrix[target_columns].drop(target_columns)

In [19]:
# Create a bar chart for correlations with Sales_Volume
fig_sales = px.bar(correlation_with_targets['Sales_Volume'].sort_values(),
                  title='Correlation with Sales_Volume',
                  labels={'value': 'Correlation Coefficient', 'index': 'Variable'},
                  color=correlation_with_targets['Sales_Volume'].sort_values(),
                  color_continuous_scale='haline',  # Blue-green color scale
                  range_color=[-1, 1])              # Fixed color range from -1 to +1

# Remove legend for cleaner visualization
fig_sales.update_layout(showlegend=False)

# Display the interactive bar chart
fig_sales.show()

In [20]:
# Create a bar chart for correlations with Stock_Quantity
fig_stock = px.bar(correlation_with_targets['Stock_Quantity'].sort_values(),
                  title='Correlation with Stock_Quantity',
                  labels={'value': 'Correlation Coefficient', 'index': 'Variable'},
                  color=correlation_with_targets['Stock_Quantity'].sort_values(),
                  color_continuous_scale='haline',  # Blue-green color scale
                  range_color=[-1, 1])              # Fixed color range from -1 to +1

# Remove legend for cleaner visualization
fig_stock.update_layout(showlegend=False)

# Display the interactive bar chart
fig_stock.show()

#### Key Findings from the Matrix

##### **1. Relationship between Sales and Stock**
The correlation between **`Stock_Quantity`** and **`Sales_Volume`** is **0.032157**, a value extremely close to zero. This indicates a **very weak positive linear relationship**. This is a crucial finding that complements the previous time-series analysis. While the time series plots showed that these variables move together in a cyclical, seasonal pattern, this matrix reveals there isn't a simple, day-to-day linear correlation. This further emphasizes that a simple linear model would fail to capture the complex, time-dependent relationship between your sales and inventory.

##### **2. Strongest Correlations**
* **`Days_For_Expiration` and `Delivery_Lag`:** These variables have a very strong positive correlation of **0.762536**. This makes intuitive sense in a supply chain context: products with a longer delivery time are likely to have a longer shelf life to compensate for the extended lead time.
* **`Inventory_Turnover_Rate` and `DOI_Inventory_Turnover`:** There is a strong negative correlation of **-0.476887** between these two. This is expected, as **Days on Inventory (DOI)** is often calculated as the reciprocal of the inventory turnover rate. A higher turnover rate means a lower number of days to sell the inventory.

#### Conclusion

The correlation matrix reinforces the finding that your supply chain's key variables do not have a strong linear relationship. This confirms that simple linear regression models would not be effective for forecasting. The most important takeaway is the near-zero correlation between `Sales_Volume` and `Stock_Quantity`, which highlights the need to use a **time-series-specific model** (like the SARIMA model discussed earlier) to capture the cyclical, seasonal, and dynamic relationship between them. The weak linear correlations across the board suggest that your project's success will depend on leveraging the time-dependent patterns rather than simple linear dependencies.

In [21]:
# Create a list of predictor variables by removing target columns from numeric columns
predictors_numeric = list(set(numeric_cols) - set(target_columns))

# Create a multi-panel histogram plot for numeric variable distributions
fig_dist = make_subplots(rows=len(predictors_numeric), cols=1,
                        subplot_titles=predictors_numeric,
                        vertical_spacing=0.05)  # Reduced spacing between subplots

# Add a histogram for each numeric predictor variable
for i, col in enumerate(predictors_numeric, 1):
    fig_dist.add_trace(go.Histogram(x=df_ts[col], name=col, nbinsx=50),  # 50 bins for detail
                      row=i, col=1)

# Adjust layout dimensions and title
fig_dist.update_layout(height=300*len(predictors_numeric),  # Dynamic height based on number of variables
                      title_text="Distribution of Numeric Predictor Variables",
                      showlegend=False,
                      bargap=0.1)  # Legend not needed for individual histograms

# Display the interactive histogram matrix
fig_dist.show()

#### **1. DOI_Inventory_Turnover & Delivery_Lag & Days_For_Expiration**
These three variables show a **highly skewed distribution**, with most of the data concentrated in a small range at the beginning. This suggests that the majority of products have a low `DOI_Inventory_Turnover`, a short `Delivery_Lag`, and a short number of `Days_For_Expiration`. This is a common pattern for fast-moving or perishable goods. A small number of products have very long tails in their distributions, which represent items that move slowly, have long lead times, or a long shelf life. These outlier values could be valuable for identifying a special class of products that require different management strategies.

#### **2. Inventory_Turnover_Rate & Reorder_Level & Reorder_Quantity**
The distributions for `Inventory_Turnover_Rate`, `Reorder_Level`, and `Reorder_Quantity` are all **relatively uniform**. This means that the values are evenly spread across their respective ranges, with no clear central tendency or skew. This suggests that the current inventory management system applies a wide variety of turnover rates, reorder levels, and reorder quantities across different products, without a single standard or most common value. This lack of a concentrated pattern could be a sign of a highly diversified product catalog or a system that has been manually adjusted over time.

#### **3. Unit_Price**
The `Unit_Price` distribution is **right-skewed**, similar to the first group of variables. Most products have a low unit price, with a few products having a much higher price. This is a typical distribution for a retail or wholesale product catalog, where low-cost items are more common than high-ticket items.

#### **Overall Conclusion**
The analysis of these distributions highlights the **diversity and complexity** of your product catalog. While some variables (like `DOI_Inventory_Turnover` and `Unit_Price`) have clear skewed patterns, others (`Reorder_Level` and `Reorder__Quantity`) are surprisingly uniform. This suggests that your inventory management is not following a one-size-fits-all approach. Your predictive models will need to account for this wide range of characteristics. The variables with highly skewed distributions may require transformation (e.g., a log transform) to improve the performance of some machine learning models.

# **Exploratory Data Analysis (EDA) Report: Intelligent System for Supply Chain Management**

### **Objective**
The primary goal of this project is to **optimize inventory and purchasing management** with the target of **reducing overstocking by 20%** within six months. This report summarizes the key findings from the exploratory data analysis (EDA) of the project's target variables: **`Stock_Quantity`** (for inventory optimization) and **`Sales_Volume`** (for demand forecasting).

---

### **1. Time Series Analysis**
The time series plots for both `Sales_Volume` and `Stock_Quantity` show significant **volatility** but no clear long-term trend. The data exhibits a strong **cyclical pattern**, which the seasonal decomposition and autocorrelation analyses confirm is due to **weekly seasonality**. Sales and stock levels rise and fall in a predictable weekly cycle. This finding is crucial because it means a time-series model can be trained to capture this pattern, providing a solid foundation for accurate forecasting.

* **Key Insight:** The relationship between sales and stock is not random; it follows a predictable, recurring pattern.

---

### **2. Autocorrelation & Granger Causality**
The autocorrelation plots for both `Sales_Volume` and `Stock_Quantity` show a very strong positive correlation at lags that are multiples of 7, confirming a dominant **weekly seasonality**.

The **Granger Causality test** revealed a critical finding:
* There is **no significant evidence** that past `Sales_Volume` values are useful for predicting future `Stock_Quantity`.
* However, there is a weak but significant link from **`Stock_Quantity` to `Sales_Volume`**.

This result indicates that the current inventory system is **not effectively using historical sales data to make proactive decisions**. Instead, it seems to be reacting to—or even driving—sales, leading to a reactive management style rather than a predictive one. This is the root cause of the overstocking problem.

---

### **3. Distribution and Correlation Analysis**
* **Distributions:** Variables such as `DOI_Inventory_Turnover`, `Unit_Price`, and `Delivery_Lag` are highly skewed, with most products falling into a low-value range. In contrast, `Reorder_Level` and `Reorder_Quantity` are more uniformly distributed. This highlights the **diversity of the product catalog** and suggests that a one-size-fits-all model won't work.
* **Correlation Matrix:** The matrix showed that there is almost **no linear correlation** between `Sales_Volume` and `Stock_Quantity` (**correlation = 0.03**). This reinforces that a simple linear regression model is not suitable for this problem and that the relationship is better explained by time-based patterns and seasonality.

### **4. Box Plot Analysis**
The box plots for `Sales_Volume` and `Stock_Quantity` show similar median values, but the presence of **outliers** in the `Stock_Quantity` data is a key concern. These outliers are a direct visual representation of the **overstocking events** that the project aims to reduce. Identifying and minimizing these instances is the central goal.

### **5. Stationarity Test**

The analysis of stationarity for both key time series variables confirms that the data is well-behaved and suitable for modeling without the need for differencing.

* **`Sales_Volume`**: The **ADF (Augmented Dickey-Fuller) and KPSS (Kwiatkowski-Phillips-Schmidt-Shin) tests both confirm that the `Sales_Volume` time series is stationary**. This is an excellent result for demand forecasting. It means the statistical properties of the series (mean, variance, and autocorrelation) remain constant over time, simplifying the modeling process and allowing for the use of forecasting models without an `I` (integrated) component.

* **`Stock_Quantity`**: Similarly, both the **ADF and KPSS tests confirm that the `Stock_Quantity` time series is also stationary**. This indicates that the inventory levels are fluctuating around a constant mean, which aligns with the finding that there is no long-term upward or downward trend.


---

### **Summary of Key Findings**
The EDA confirms that your data is well-suited for a time-series forecasting approach. The key takeaways are:
1.  **Stationarity:** Both `Sales_Volume` and `Stock_Quantity` are stationary, which simplifies modeling.
2.  **Seasonality:** Strong weekly seasonality is the primary pattern in the data.
3.  **Reactive System:** The current inventory system is not effectively using sales history to make proactive decisions, leading to overstocking.

### **Next Steps and Recommendations**
To achieve the 20% overstocking reduction, the project must shift to a proactive, data-driven approach:
* **Build a Seasonal Forecasting Model:** Develop a model (e.g., SARIMA) that leverages the weekly seasonality to accurately predict `Sales_Volume`.
* **Integrate Forecasts:** Use the model's output to directly inform and automate inventory decisions, establishing the missing link from sales to stock.
* **Target Outliers:** Focus on specific inventory management policies to address the root causes of the identified overstocking outliers.

